In [5]:
!pip install -U langchain-google-genai

In [6]:
!pip install newspaper3k


In [7]:
!pip install lxml[html_clean]


In [8]:
from google.colab import userdata
api_key=userdata.get('GEMINI_API_KEY')

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(api_key=api_key,model="gemini-2.5-flash")

In [14]:
import os
import json
import datetime
import textwrap
from newspaper import Article
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage
from IPython.display import display, Markdown

# ✅ Initialize model safely
try:
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0.2,
        google_api_key=api_key
    )
except Exception as e:
    raise SystemExit(f"❌ Failed to initialize LLM: {e}")


def to_markdown(text):
    """Formats text as Markdown for display."""
    text = text.replace("•", "-")
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))


def extract_article(url):
    """Extracts article content, title, and publication date using newspaper3k."""
    try:
        article = Article(url)
        article.download()
        article.parse()

        title = article.title or "Untitled Article"
        text = article.text or ""
        date = (
            article.publish_date.strftime("%d %B %Y")
            if article.publish_date
            else datetime.date.today().strftime("%d %B %Y")
        )

        if not text.strip():
            raise ValueError("Article text is empty.")
        return title, text, date

    except Exception as e:
        print(f" Error extracting article: {e}")
        return None, None, datetime.date.today().strftime("%d %B %Y")


def summarize_article(text):
    """Summarizes the article using Gemini 2.5 Flash."""
    try:
        prompt = (
            "Summarize the following article into 5 concise bullet points. "
            "Keep the language clear and simple.\n\n"
            f"{text}"
        )
        response = llm.invoke([HumanMessage(content=prompt)])
        return response.content.strip() if response and response.content else "Summary not available."
    except Exception as e:
        return f" Error during summarization: {e}"


def classify_article_type(text):
    """Classifies the article type using Gemini 2.5 Flash."""
    try:
        prompt = (
            "Classify this article into one of the following categories: "
            "technology, entertainment, sports, financial, health, or general. "
            "Return only the category name.\n\n"
            f"{text}"
        )
        response = llm.invoke([HumanMessage(content=prompt)])
        return response.content.strip() if response and response.content else "Unknown"
    except Exception as e:
        return f" Error during classification: {e}"


def save_to_json(date, summary, article_type):
    """Saves data to a JSON file safely and returns updated JSON."""
    entry = {
        "date": date,
        "summary": summary,
        "articleType": article_type
    }

    try:
        if os.path.exists("summaries.json"):
            with open("summaries.json", "r", encoding="utf-8") as f:
                try:
                    data = json.load(f)
                except json.JSONDecodeError:
                    data = []
        else:
            data = []

        data.append(entry)
        with open("summaries.json", "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=4)

        return data

    except Exception as e:
        print(f" Error saving JSON: {e}")
        return [entry]


def ask_question_about_json(json_data, question):
    """Uses Gemini 2.5 Flash to answer a question based on the JSON summary."""
    try:
        prompt = (
            f"Here is a JSON news summary:\n{json.dumps(json_data, indent=4)}\n\n"
            f"Question: {question}"
        )
        response = llm.invoke([HumanMessage(content=prompt)])
        return response.content.strip() if response and response.content else "No response from model."
    except Exception as e:
        return f" Error during question answering: {e}"


def main():
    print("---  News Summarization Tool ---\n")

    url = input("\nEnter the news article URL: ").strip()
    if not url:
        print(" URL cannot be empty.")
        return

    # Extract article
    print("\n Extracting article content...")
    title, content, date = extract_article(url)

    if not content:
        print(" Failed to extract content. Please check the URL.")
        return

    display(to_markdown(f"**Title:** {title}\n**Date:** {date}"))

    # Summarize
    print("\n Generating summary...")
    summary = summarize_article(content)
    display(to_markdown(f"**Summary:**\n{summary}"))

    # Classify
    print("\n Classifying article type...")
    article_type = classify_article_type(content)
    display(to_markdown(f"**Article Type:** {article_type}"))

    # JSON output (cleaned format)
    json_data = save_to_json(date, summary, article_type)
    formatted_json = json.dumps(json_data[-1], indent=4, ensure_ascii=False)
    display(Markdown(f"###  **JSON Summary Saved Successfully:**\n```json\n{formatted_json}\n```"))

    # Ask a question
    question = input("\n Ask a question about this summary (or press Enter to skip): ").strip()
    if question:
        answer = ask_question_about_json(json_data[-1], question)
        display(to_markdown(f"**Answer:**\n{answer}"))
    else:
        print("No question asked. Process completed successfully ")


if __name__ == "__main__":
    main()



---  News Summarization Tool ---


Enter the news article URL: https://levelup.gitconnected.com/37-lessons-from-my-7-years-in-software-engineering-ad2365e3722a

 Extracting article content...


> **Title:** 37 Lessons From My 7 Years in Software Engineering
> **Date:** 15 June 2025


 Generating summary...


> **Summary:**
> Here's a summary of the article's introduction in 5 concise bullet points:
> 
> *   The author reflects on their seven-year journey in software engineering, which began in February 2018 as a fresh graduate.
> *   Initially, they felt nervous, clueless, and had big dreams for their new career.
> *   Over the past seven years, their experience has been a mix of highs and lows, leading to significant personal growth.
> *   This journey included excitement, burnout, imposter syndrome, big wins, and late-night debugging sessions.
> *   The article promises to share 37 unique lessons learned during this time, which are not taught in traditional degrees or tutorials.


 Classifying article type...


> **Article Type:** Technology

###  **JSON Summary Saved Successfully:**
```json
{
    "date": "15 June 2025",
    "summary": "Here's a summary of the article's introduction in 5 concise bullet points:\n\n*   The author reflects on their seven-year journey in software engineering, which began in February 2018 as a fresh graduate.\n*   Initially, they felt nervous, clueless, and had big dreams for their new career.\n*   Over the past seven years, their experience has been a mix of highs and lows, leading to significant personal growth.\n*   This journey included excitement, burnout, imposter syndrome, big wins, and late-night debugging sessions.\n*   The article promises to share 37 unique lessons learned during this time, which are not taught in traditional degrees or tutorials.",
    "articleType": "Technology"
}
```


 Ask a question about this summary (or press Enter to skip): summarize the article in few words


> **Answer:**
> 7 years of software engineering lessons.